# Interface Demonstration Notebook

A notebook to demonstrate an interface with some of the features of the Probability of Failure Model

Author: gavin.treseder@essentialenergy.com.au

In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

import copy

from pof.component import Component
from pof.failure_mode import FailureMode
from interface.layouts import *

In [2]:
comp = Component().set_demo()
fm = FailureMode().set_demo()

In [3]:
# Component Tests

mcl = make_component_layout(comp, prefix='comp-')

In [4]:
graph_limit =  dbc.InputGroup(
    [
        dbc.InputGroupAddon(
            [
                dbc.Checkbox(id='graph_y_limit_active', checked=True),
                
            ],
            addon_type="prepend"
        ),
        dbc.Label("Graph Y Limit"),
        dbc.Input(
            type="number",
            id= 'graph_y_limit',
            #value = ,
            debounce=True,
        ),
    ],
)

graph_inputs = [Input('graph_y_limit_active', 'checked'), Input('graph_y_limit', 'value')]

In [6]:

# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
app.layout = html.Div([
    html.Div(children='Test Output', id='test_output'),
    graph_limit,
    dcc.Graph(id="maintenance_strategy"),
    mcl,
])

sep= '-'
collapse_ids = []
for comp_name, comp in {'comp' : comp}.items():
    comp_prefix = comp_name + sep
    collapse_ids = collapse_ids + [comp_name]
    # Failure modes
    fm_prefix = comp_prefix + 'failure_mode' + sep
    for fm_name, fm in comp.fm.items():
        collapse_ids = collapse_ids + [fm_prefix + fm_name]
        # Tasks
        task_prefix = fm_prefix + fm_name + sep + 'task' + sep
        for task in fm.tasks:
            collapse_ids = collapse_ids + [task_prefix + task]


@app.callback(
    #Output("test_output", "children"),
    [Output(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
    [Input(f"{prefix}-collapse-button", "n_clicks") for prefix in collapse_ids],
    [State(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
)
def toggle_collapses(*args):
    ctx = dash.callback_context

    state_id = ""
    collapse_id = ctx.triggered[0]['prop_id'].split('.')[0].replace('-collapse-button','')
    if collapse_id in collapse_ids: #TODO change to is not None

        state_id = collapse_id + '-collapse.is_open'
        ctx.states[state_id] = not ctx.states[state_id]

    is_open = tuple(ctx.states.values())

    return is_open

ms_fig_update = comp.get_dash_ids(prefix ='comp-')

@app.callback(
    Output("maintenance_strategy", "figure"), 
    graph_inputs + [Input(dash_id,"checked") if 'active' in dash_id else Input(dash_id,"value") for dash_id in ms_fig_update]
)
def update_maintenance_strategy(graph_y_limit_active, graph_y_limit, *args):

    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
    value = ctx.triggered[0]['value']

    #Temp utnil mutliple failure modes are set up
    dash_id = dash_id.replace('comp-', "")

    # Update the model
    comp.dash_update(dash_id, value)

    comp.mc_timeline(t_end=200, n_iterations=100)
    df = comp.expected_risk_cost_df(t_end=200)

    fig = px.area(
        df,
        x="time",
        y="cost_cumulative",
        color="task",
        line_group='failure_mode',
        title="Maintenance Strategy Costs",
    )

    if graph_y_limit_active:
        fig.update_yaxes(range=[0, graph_y_limit])

    return fig

@app.callback(
    Output("test_output", "children"),
    [Input('comp-active', 'checked')]
)
def test(task_value):
    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
    value = ctx.triggered[0]['value']

    return "value" + str(value)

#Execute
app.run_server(mode='inline', debug=True)

In [10]:
comp.fm['random'].__dict__

100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100.]),
   'external_diameter': array([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
          100., 100., 100., 100., 100., 100., 1

In [7]:
comp.fm['slow_aging'].active

True

NameError: name 'prefix' is not defined

In [8]:
dash_id = 'task-inspection-active'
dash_id = dash_id.replace(next_id + sep, "")
task_name= dash_id.split(sep)[0]
dash_id = dash_id.replace(next_id + sep, "")
dash_id

'inspection-active'

In [13]:
task.active

False

In [12]:
fm.tasks['inspection'].__dict__['active'] = False

In [11]:
fm.tasks['inspection'].__dict__

{'name': 'unknown',
 'activity': 'inspection',
 'trigger': 'time',
 'active': True,
 'trigger_comp': '1.1.2',
 '_package': NotImplemented,
 '_impacts_parent': NotImplemented,
 '_impacts_children': False,
 'cost': 50,
 'labour': 'trade',
 'spares': 'pole',
 'equipment': 'ewp',
 'consequence': None,
 'p_effective': 0.91,
 'triggers': {'condition': {'wall_thickness': {'lower': 0, 'upper': 90}},
  'state': {'initiation': True}},
 'impacts': {'condition': {}, 'state': {'detection': True}},
 'component_reset': False,
 'state': 'up',
 't_completion': [5,
  10,
  15,
  20,
  25,
  30,
  35,
  40,
  45,
  50,
  55,
  60,
  65,
  70,
  75,
  80,
  85,
  90,
  95,
  100,
  105,
  110,
  115,
  120,
  125,
  130,
  135,
  140,
  145,
  150,
  155,
  160,
  165,
  170,
  175,
  180,
  185,
  190,
  195,
  200,
  5,
  10,
  15,
  20,
  25,
  30,
  35,
  40,
  45,
  50,
  55,
  60,
  65,
  70,
  75,
  80,
  85,
  90,
  95,
  100,
  105,
  110,
  115,
  120,
  125,
  130,
  135,
  140,
  145,
  150,
 